## The Experiment and Sequencing Data

This notebook serves as a guide to analyzing the mutations found from sequencing data from the NS001 evolution in a platereader experiment.

DNA samples were sequenced on an Illumina MiSeq. Each sample was analyzed for mutations against a modified MG1655 sequence (modified to match consensus sequence of NS001 from aggregate of 3 sequenced samples) using [BreSeq](http://barricklab.org/twiki/pub/Lab/ToolsBacterialGenomeResequencing/documentation/).

BreSeq output consists of a collection of html and plaintext files for each sample sequenced that contain information on mutations detected in the sequenced sample compared to a reference sequence as well as regions of missing coverage and other data. The html files are nice for visual inspection, but when it comes to doing any serious analysis comparing samples, this format is very hard to work with. So I moved the necessary data on mutations (currently only SNP mutations which are ~95% of the mutations) for all samples into SQL tables. There is fairly small amount of data once you're not lugging around all of the sequencing data (a little over a gigabyte per sample) which means everything fits into a single [SQLite](https://www.sqlite.org/index.html) file. This file is currently NS001_evolved_mutations_copy.db . This database choice makes sense as long as the total size of all the mutation data is something like tens or hundreds of gigabytes. The ~40 samples currently lead to an SQLite file that's only 324 kilobytes.

You can inspect the SQLite file with [DB Browser](https://sqlitebrowser.org/) which can also export the SQL tables to .csv. You can then load the .csv into any tool you like (for example, excel). But I recommend using Python to work with the database file itself since I've already invested time into making that less painful using [SQLAlchemy](https://docs.sqlalchemy.org/en/13/) to map the SQL tables to Python objects. The file that contains this mapping is SequenceDataORM.py.

## Working with the Data in Python

First we need to import the necessary SQLAlchemy libraries for loading the sql file with the sequence data and mapping it to objects, NumPy and pandas for analyzing data, SequenceDataORM which contains the code that maps our SQL tables to Python objects, and matplotlib and seaborn for visualization.

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import numpy as np
import pandas as pd

import SequenceDataORM as sqd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Next we load the data using create_engine from SQLAlchemy.

In [2]:
engine = create_engine('sqlite:///NS001_evolved_mutations_copy8.db', echo=False) # our database connection

Let's check what tables are in the database we've loaded.

In [3]:
engine.table_names()

['del_evidence',
 'del_mutations',
 'dna_samples',
 'genes',
 'ins_evidence',
 'ins_mutations',
 'mc_evidence',
 'mob_evidence',
 'mob_mutations',
 'mutation_conditions',
 'snp_evidence',
 'snp_mutations',
 'strains',
 'wells']

There are seven tables making up the database. Because mutations that occur in an ancestor also usually show up in the parent, there is a many-to-many mapping between DNA samples of evolved NS001 (from different wells and different days during the course of evolution) and mutations that occur. This means that the SQL tables involved are a little complicated. The tables snp_mutations and dna_samples hold all the mutations found by breseq and names of sequenced dna_samples respectively. The table snp_evidence serves as a link between mutations and samples they occurred in.

Let's look at what each table looks like by looking at the top few rows. To do this, we'll use pandas to convert SQL tables into a pandas dataframe using the <code>read_sql</code> function from pandas. The <code>read_sql</code> method requires two arguments, an SQL query and a database connection. The engine variable we defined above is the database connection.

Then we'll display the top few rows of each table using the <code>head</code> method of the dataframe we just got from our SQL query.

## Explanation of the tables

The wells table just acts as a consistency check so that when I inserted what well a strain was grown in, the SQL table wouldn't accept 'H10'. I only used a 48-well plate, and not all 48 wells.

In [4]:
df_wells = pd.read_sql('SELECT * FROM wells', engine)
df_wells.head()

,position
0,A1
1,A2
2,A3
3,A4
4,B2


The mutation_conditions table both acts as a consistency check to make sure I always insert the mutation conditions in the same way I named them in the paper, and contains the mutation rate estimates and confidence intervals from the paper so I don't have to keep looking them up.

In [5]:
df_mutation_conditions = pd.read_sql('SELECT * FROM mutation_conditions', engine)
df_mutation_conditions.head()

,name,mu_est,mu_lowCI,mu_hiCI
0,Hi,2.200000e-07,1.600000e-07,2.900000e-07
1,HiMid,4.100000e-08,2.600000e-08,5.900000e-08
2,Mid,1.400000e-08,6.400000e-09,2.500000e-08
3,LoMid,3.800000e-09,1.200000e-09,7.400000e-09
4,Lo,1.700000e-09,4.100000e-10,3.600000e-09


The strains table contains the name of each strain, its last frozen and sequenced ancestor, the day in the platereader evolution experiment that strain came from (day 0 is actually the first day so day 24 is the 25th day of the experiment), the well the strain was grown in during the evolution experiment (if applicable), and the mutation rate condition (if applicable).

In [6]:
df_strains = pd.read_sql('SELECT * FROM strains', engine)
df_strains.head()

,name,ancestor,day_in_platereader,well,mutation_rate_condition
0,NS001,None,NaN,None,None
1,Hi1t1,NS001,24.0,A1,Hi
2,Hi2t1,NS001,24.0,A2,Hi
3,Hi3t1,NS001,24.0,A3,Hi
4,Hi4t1,NS001,24.0,A4,Hi


The dna_samples table contains info on which sequence sample corresponds to which strain. It's generally obvious since the name is almost identical and all of my strains but one were only sequenced once. There are multiple samples corresponding to the strain NS001 although they aren't all inserted yet. So that's why there's a separate table for samples from strains. 

In [7]:
df_dna_samples = pd.read_sql('SELECT * FROM dna_samples', engine)
df_dna_samples.head()

,name,strain
0,Aggregate_NS001_Ancestors,NS001
1,Hi1t1_S1,Hi1t1
2,Hi1t2_S1,Hi1t2
3,Hi2t1_S1,Hi2t1
4,Hi2t2_S1,Hi2t2


The genes table contains gene names, their descriptions from BreSeq (which themselves come from annotated file I pulled from NCBI for MG1655, modified a couple redundant annotations so BreSeq wouldn't error out, and was then further modified by BreSeq to match the consensus of NS001 sequencing), and whether or not that region is coding region (the intergenic column). If the intergenic column is a 0, then the region is a coding region. If it's a 1, then it's a noncoding region. When there is a | in the name of the gene, then either that DNA segment is part of two genes (if intergenic is 0) or the region is between those two coding genes (if intergenic is 1).

In [8]:
df_genes = pd.read_sql('SELECT * FROM genes', engine)
df_genes.head()

,name,description,intergenic,start,end,strand
0,murC|ddlB,UDP-N-acetylmuramate--L-alanine ligase|D-alani...,0,NaN,NaN,NaN
1,yadM,putative fimbrial protein YadM,0,152243.0,152812.0,0.0
2,htrE,putative fimbrial usher protein HtrE,0,152829.0,155426.0,0.0
3,betB,betaine aldehyde dehydrogenase,0,326485.0,327957.0,0.0
4,brnQ,branched chain amino acid transporter BrnQ,0,418814.0,420133.0,1.0


The snp_mutations table is one of the meatier tables. It includes all mutations found in all strains by sequencing.

chr_position is the position of the mutation in the chromosome (relative to the ancestral sequence), ref_base is the original nucleotide in the ancestor, new_base is the nucleotide of the mutation, and gene is self explanatory (and its description can be found in the genes table\*). NaN's correspond to values that don't make sense for a particular region. For example, coding regions don't have position measured by intergenic distance from a coding gene (intergenic_left and intergenic_right), and noncoding regions don't change ref_aa (reference amino acid in the ancestor) to a new_aa (the new amino acid produced due to the mutation).

\* the gene column is what is called a foreign key in SQL

In [9]:
df_snp_mutations = pd.read_sql('SELECT * FROM snp_mutations', engine)
df_snp_mutations.head()

,chr_position,ref_base,new_base,gene,gene_pos,ref_aa,new_aa,intergenic_left,intergenic_right
0,102236,A,G,murC|ddlB,1472|4,D|T,G|A,NaN,NaN
1,152255,G,A,yadM,558,F,F,NaN,NaN
2,155269,T,C,htrE,158,Y,C,NaN,NaN
3,327888,A,G,betB,70,F,L,NaN,NaN
4,419369,T,C,brnQ,556,F,L,NaN,NaN


The snp_evidence table links together sequenced DNA_samples and mutations. Each row corresponds to a sample and a mutation found in it. Samples must be in the dna_samples table and chr_position, ref_base, and new_base must correspond to a row in the snp_mutations table. The frequency column is the estimated frequency of that mutation in that sample according to breseq. The remaining columns are various sequencing statistics for the sample. The most interesting ones are probably ref_cov, new_cov, and total_cov. These numbers separated by a slash are the number of reads from each strand of the chromosome. ref_cov is how often the ref_base in the reference sequence was found at that position in the sample, new_cov is how often the new_base was found in the sample, and total_cov is the total coverage of that location in the sample. Sometimes it's slightly larger than ref_cov + new_cov because a minor variant was read during sequencing. That could be the result of a sequencing error (very likely) or another polymorphism at the same location (very unlikely).

In combination with the SequenceData ORM file, this table gives us an easy way to find all mutations in a particular sample and all samples that have a given mutation. I'll show how to work with the data using Python ojbects next.

In [10]:
df_snp_evidence = pd.read_sql('SELECT * FROM snp_evidence', engine)
df_snp_evidence.head()

,sample,chr_position,ref_base,new_base,frequency,bias_e_value,bias_p_value,consensus_score,fisher_strand_p_value,ref_cov,new_cov,total_cov
0,Aggregate_NS001_Ancestors,102236,A,G,0.178867,4641650.0,1.000000,593.1,1.000000,96/106,21/24,117/131
1,Aggregate_NS001_Ancestors,152255,G,A,0.067267,4641110.0,0.999883,190.7,1.000000,31/24,2/2,33/26
2,Aggregate_NS001_Ancestors,155269,T,C,0.444699,4641650.0,1.000000,46.7,1.000000,28/28,22/23,50/51
3,Aggregate_NS001_Ancestors,327888,A,G,0.096040,4637600.0,0.999127,518.3,1.000000,69/86,7/10,82/96
4,Aggregate_NS001_Ancestors,419369,T,C,0.067444,1004830.0,0.216481,580.2,0.162411,81/87,9/4,91/91


## The Object-Relational Mapping between Python Objects and the SQL tables

To work with the database using Python objects, we first need to open a session using SQLAlchemy's <code>sessionmaker</code> function and bind it to the engine object we made when we loaded the data

In [11]:
session = sessionmaker(bind=engine)() # the session object is how we make queries through sqlalchemy

Each table in the SQL database has a corresponding Python class in the SequenceDataORM.py file (which we imported as sqd). Each row of the table corresponds to a python object. For the tables,  'wells', 'mutation_conditions', 'strains', 'dna_samples', 'genes', 'snp_mutations', and 'snp_evidence' the corresponding classes are <code>Well</code>, <code>MutationCondition</code>, <code>Strain</code>, <code>DNA_Sample</code>, <code>Gene</code>, <code>SNP_Mutation</code>, and <code>SNP_Evidence</code>.
 
We can issue a query to the database to return a table as a list of objects using the <code>query</code> method of our session. For example, to see a list of SNP_Mutations

In [12]:
all_SNPs = [snp for snp in session.query(sqd.SNP_Mutation)]
for i in range(5):
    print(all_SNPs[i])

<SNP_Mutation(chr_position=102236, ref_base=A, new_base=G, gene=murC|ddlB)>
<SNP_Mutation(chr_position=152255, ref_base=G, new_base=A, gene=yadM)>
<SNP_Mutation(chr_position=155269, ref_base=T, new_base=C, gene=htrE)>
<SNP_Mutation(chr_position=327888, ref_base=A, new_base=G, gene=betB)>
<SNP_Mutation(chr_position=419369, ref_base=T, new_base=C, gene=brnQ)>


For convenience, when printed, the objects always won't display all the data from their row, but you can access it with Python's . notation for attributes of objects. The column names in the SQL table are also attribute names of each Python object. For example,

In [13]:
first_SNP = all_SNPs[0]
print('The first SNP is at chromosomal position {0},'
      ' which is the {1} nucleotide of the gene {2}.\n'
      'This changed the amino acid {3} to {4}.'
      .format(first_SNP.chr_position, first_SNP.gene_pos, first_SNP.gene, first_SNP.ref_aa, first_SNP.new_aa))

The first SNP is at chromosomal position 102236, which is the 1472|4 nucleotide of the gene murC|ddlB.
This changed the amino acid D|T to G|A.


There are also some convenience attributes that Python objects associated with this data have that aren't directly in the SQL table, but are easily calculated on the fly. For example, a mutation is synonymous if it doesn't change the amino acide coded for in a gene. We can check if a mutation is synonomous with the <code>synonymous</code> attribute

In [14]:
first_SNP.synonymous

False

Technically, whether or not a mutation is synonymous depends on whether or not it is in a coding region. Since noncoding regions don't change any amino acids, we've defined them as synonymous. We might be curious how many of all the mutations found are not synonymous

In [15]:
are_synonymous = [snp.synonymous for snp in all_SNPs]
N_nonsynonymous = np.sum(np.logical_not(are_synonymous)) # this counts how many Falses there were for synonymous
print('There are {0} nonsynonymous mutations out of {1} mutations total'.format(N_nonsynonymous, len(are_synonymous)))

There are 543 nonsynonymous mutations out of 937 mutations total


The nicest convenience attribute is that Python SNP_mutation objects know what dna samples had those mutations and Python DNA_Sample objects know what mutations they have. In SQL this would involve using some <code>groupby</code> statements on the evidence table or <code>join</code>s between tables or something. In Python, these are just attributes of those objects.

In [16]:
print('The samples that had the mutation {0} were: {1}'.format(first_SNP, first_SNP.samples))

The samples that had the mutation <SNP_Mutation(chr_position=102236, ref_base=A, new_base=G, gene=murC|ddlB)> were: ['Aggregate_NS001_Ancestors', 'Ancestor_S1', 'Ancestor_S2', 'Ancestor_S3', 'Hi1t1_S1', 'Hi1t2_S1', 'HiMid2t1_S1', 'HiMid2t2_S1', 'HiMid3t2_S1', 'HiMid4t1_S1', 'HiMid4t2_S1', 'Lo1t1_S1', 'Lo2t1_S1', 'Lo2t2_S1', 'Lo3t1_S1', 'Lo4t1_S1', 'Lo4t2_S1', 'LoMid1t1_S1', 'LoMid1t2_S1', 'LoMid2t1_S1', 'LoMid3t1_S1', 'LoMid3t2_S1', 'LoMid4t1_S1', 'LoMid4t2_S1', 'Mid1t1_S1', 'Mid2t1_S1', 'Mid3t1_S1']


Similarly, given a sample, we can retrieve a list of the SNPs detected in it.

In [17]:
all_DNA_samples = [sample for sample in session.query(sqd.DNA_Sample)]
first_DNA_sample = all_DNA_samples[0]
for i in range(5):
    print(all_DNA_samples[i])

<DNA_Sample(name=Aggregate_NS001_Ancestors, strain=NS001)>
<DNA_Sample(name=Hi1t1_S1, strain=Hi1t1)>
<DNA_Sample(name=Hi1t2_S1, strain=Hi1t2)>
<DNA_Sample(name=Hi2t1_S1, strain=Hi2t1)>
<DNA_Sample(name=Hi2t2_S1, strain=Hi2t2)>


The mutations are given as a tuple of their chromosome position, reference nucleotide, and mutation nucleotide. This uniquely identifies a mutation.

In [18]:
first_DNA_sample.snp_mutations

[(102236, 'A', 'G'), (152255, 'G', 'A'), (155269, 'T', 'C'), (327888, 'A', 'G'), (419369, 'T', 'C'), (547336, 'C', 'T'), (735938, 'A', 'G'), (1055424, 'A', 'T'), (1171667, 'A', 'G'), (1541878, 'A', 'G'), (1733676, 'T', 'C'), (1851805, 'T', 'C'), (2747257, 'G', 'A'), (3089230, 'T', 'C'), (3437557, 'A', 'G'), (3629956, 'T', 'C'), (3688257, 'A', 'G'), (4091182, 'C', 'T'), (4295832, 'T', 'C'), (4296058, 'C', 'T'), (4296153, 'T', 'C'), (4296188, 'A', 'G'), (4296189, 'A', 'C'), (4397411, 'G', 'C')]

SNP_Mutations have an attribute <code>mutation</code> that lets us compare a tuple of chromosome position, reference nucleotide, and mutation nucleotide to a Python object to see if they correspond

In [19]:
first_SNP.mutation == (102236, 'A', 'G')

True

In [20]:
first_SNP.mutation == (12345, 'X', 'Y')

False